In [9]:
!pip install yfinance torch transformers
!pip install transformers accelerate bitsandbytes





[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip


In [10]:
import yfinance as yf
import pandas as pd

data = yf.download("AAPL", start="2015-01-01", end="2025-01-01", interval="1d")

data['return'] = data['Close'].pct_change()
data = data.dropna()

data.head()


/var/folders/54/vyhscs1x4378wr1lbytk_fj00000gn/T/ipykernel_80868/2273444151.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("AAPL", start="2015-01-01", end="2025-01-01", interval="1d")
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume,return
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,
Date,,,,,,
2015-01-05,23.577576,24.110152,23.391175,24.030265,257142000,-0.028172
2015-01-06,23.579796,23.839426,23.218087,23.641929,263188400,0.000094
2015-01-07,23.910433,24.010290,23.677430,23.788384,160423600,0.014022
2015-01-08,24.829128,24.886824,24.121246,24.238858,237458000,0.038422
2015-01-09,24.855751,25.130915,24.456319,25.002208,214798000,0.001072


In [11]:
import numpy as np

window_size = 7  # how many previous days to use
returns = data['return'].values

X, y = [], []
for i in range(window_size, len(returns) - 1):
    X.append(returns[i - window_size:i])
    y.append(returns[i])
X, y = np.array(X), np.array(y)


In [12]:
import torch
import torch.nn as nn
from transformers import GPT2Config, GPT2Model

class StockLLM(nn.Module):
    def __init__(self, hidden=32, n_layer=2, n_head=2):
        super().__init__()
        config = GPT2Config(
            n_embd=hidden,
            n_layer=n_layer,
            n_head=n_head,
            vocab_size=1_000,   # dummy
            n_positions=128,
            n_ctx=128,
        )
        self.backbone = GPT2Model(config)
        self.input_proj = nn.Linear(1, hidden)
        self.reg_head = nn.Linear(hidden, 1)

    def forward(self, x):
        # x: [batch, seq_len]
        x = x.unsqueeze(-1)  # [batch, seq_len, 1]
        emb = self.input_proj(x)
        out = self.backbone(inputs_embeds=emb).last_hidden_state
        pred = self.reg_head(out[:, -1, :])
        return pred.squeeze(-1)
    


In [13]:
from torch.utils.data import TensorDataset, DataLoader

# Train/test split
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Convert to tensors
train_ds = TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                         torch.tensor(y_train, dtype=torch.float32))
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)

model = StockLLM()
opt = torch.optim.AdamW(model.parameters(), lr=1e-4)
loss_fn = nn.MSELoss()

for param in model.backbone.parameters():
    param.requires_grad = False

optimizer = torch.optim.AdamW(
    list(model.input_proj.parameters()) + list(model.reg_head.parameters()),
    lr=1e-3
)

def train_model(num_epochs):
  for epoch in range(num_epochs):
      model.train()
      total_loss = 0
      total_mse = 0

      for xb, yb in train_dl:
          optimizer.zero_grad()
          pred = model(xb)
          loss = loss_fn(pred, yb)
          loss.backward()
          optimizer.step()
          total_loss += loss.item()



      avg_loss = total_loss / len(train_dl)
      print(f"Epoch {epoch+1}: "
              f"MSE={avg_loss}")

train_model(5)
print()

def unfreeze_layers(model, n_unfreeze):
    layers = list(model.backbone.h)  # transformer blocks
    for layer in layers[-n_unfreeze:]:
        for param in layer.parameters():
            param.requires_grad = True

unfreeze_layers(model, n_unfreeze=2)
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)

train_model(5)
print()

for param in model.backbone.parameters():
    param.requires_grad = True

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-7)
train_model(5)
print()

Epoch 1: MSE=0.08815694768868741
Epoch 2: MSE=0.026365512568089697
Epoch 3: MSE=0.023879536013636325
Epoch 4: MSE=0.01896715925503818
Epoch 5: MSE=0.016463146108897434

Epoch 1: MSE=0.014334006722839106
Epoch 2: MSE=0.015238181000486725
Epoch 3: MSE=0.014048440647976739
Epoch 4: MSE=0.013487857286005265
Epoch 5: MSE=0.01236210385751393

Epoch 1: MSE=0.013052140185165973
Epoch 2: MSE=0.013793888989658583
Epoch 3: MSE=0.012487816905218458
Epoch 4: MSE=0.012011347094639427
Epoch 5: MSE=0.012928040535558783



In [14]:
model.eval()
with torch.no_grad():
    X_test_t = torch.tensor(X_test, dtype=torch.float32)
    y_test_t = torch.tensor(y_test, dtype=torch.float32)
    preds = model(X_test_t)

mse = ((preds - y_test_t)**2).mean().item()
direction_acc = ((torch.sign(preds) == torch.sign(y_test_t)).float().mean().item())

print(f"MSE: {mse:.6f}")
print(f"Directional Accuracy: {direction_acc*100:.2f}%")


MSE: 0.000219
Directional Accuracy: 56.37%


In [15]:
torch.save(model.state_dict(), "model.pt")  # only save weights
